In [1]:
import pandas as pd
import numpy as np
import csv
import xgboost as xgb
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,mean_absolute_error
import pickle

In [ ]:
with open('/content/dailymonthly.csv', encoding='utf-8-sig') as file_obj:
    # Create reader object by passing the file object to DictReader method
    reader_obj = csv.DictReader(file_obj)

    # Open a file to save the results
    with open("/content/XGBoost_IMD_SPEI.csv", "w+") as f:
        # Define a csv writer object
        writer = csv.writer(f)
        writer.writerow(['Loc', 'mse', 'rmse', 'mae'])

        # Iterate over each row in the csv file using reader object
        for row in reader_obj:
            data = row['daily']
            print(row)

            imd = pd.read_csv('/content/IMD/' + data + '.csv').drop(['Unnamed: 0', 'X', 'Year', 'Month'], axis=1)
            imd = imd.replace([np.inf, -np.inf], np.nan)

            X = imd.iloc[5:, 0:4]
            Y = imd.iloc[5:, 4:5]

            imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
            imputer.fit(Y)
            Y = imputer.transform(Y)
            X['Spei6'] = Y
            Y = X.iloc[:, 3]
            Ya = Y.iloc[10:]

            X_train = np.zeros((544, 20))
            X_test = np.zeros((200, 20))
            for i in range(544):
                X_train[i, :] = Y[i:i + 20]
            for i in range(200):
                X_test[i, :] = Y[i + 544:i + 544 + 20]

            X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
            Y_train = np.array(Ya.iloc[0:544])
            Y_train = Y_train.reshape(Y_train.shape[0], 1, 1)

            X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
            Y_test = np.array(Ya.iloc[544:])
            Y_test = Y_test.reshape(Y_test.shape[0], 1, 1)

            X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

            params={
                'objective': 'reg:squarederror',
                'alpha': 0.5,
                'lambda': 0.5,
                'subsample': 0.8,
                'colsample_bytree': 0.8,
                'learning_rate': 0.1,
                'max_depth': 5,
                'n_estimators': 100
               }

            model = xgb.XGBRegressor(
                **params
                )
            model.fit(
                X_train,
                Y_train
                )
            Y_pred = model.predict(X_test)

            with open("/content/model/"+data+'.h5', "wb") as model_file:
              pickle.dump(model, model_file) #store the model file for using for CMIP6 data for validation.

            mse = mean_squared_error(Y_test, Y_pred)
            rmse = np.sqrt(mse)
            mae = mean_absolute_error(Y_test, Y_pred)


            print("XGBoost model evaluation report:")
            print("----------------------------")
            print("Mean Squared Error:",mse)
            rmse = round((np.sqrt(mse)) , 5)
            print("Root Mean Squared Error:",rmse)
            print("Mean Absolute Error:",mae)
            print("\n \n")

            # Write the results to the file
            writer.writerow([data, mse, rmse, mae])

In [ ]:
!zip -r ./model.zip ./model/
from google.colab import files
files.download("/content/model.zip")

  adding: model/ (stored 0%)
  adding: model/Dausa.h5 (deflated 76%)
  adding: model/Bharatpur.h5 (deflated 76%)
  adding: model/Sri Ganganagar.h5 (deflated 76%)
  adding: model/Pali.h5 (deflated 76%)
  adding: model/Bundi.h5 (deflated 76%)
  adding: model/Banswara.h5 (deflated 77%)
  adding: model/Sikar.h5 (deflated 76%)
  adding: model/Kekri.h5 (deflated 77%)
  adding: model/Khairthal Tijara.h5 (deflated 76%)
  adding: model/Ajmer.h5 (deflated 76%)
  adding: model/Sawai Madhopur.h5 (deflated 77%)
  adding: model/Udaipur.h5 (deflated 76%)
  adding: model/Deeg.h5 (deflated 76%)
  adding: model/Tonk.h5 (deflated 77%)
  adding: model/Gangapur.h5 (deflated 76%)
  adding: model/Kota.h5 (deflated 77%)
  adding: model/Barmer.h5 (deflated 76%)
  adding: model/Sirohi.h5 (deflated 77%)
  adding: model/Rajsamand.h5 (deflated 76%)
  adding: model/Phalodi.h5 (deflated 76%)
  adding: model/Jaipur.h5 (deflated 76%)
  adding: model/Anupgarh.h5 (deflated 76%)
  adding: model/Churu.h5 (deflated 76%)
  

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>